In [1]:
%pip install pytorch_forecasting
%pip install bashlex

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.5/69.5 kB 10.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
dataset = []

https://gitlab.ics.muni.cz/muni-kypo-trainings/datasets/commands


In [3]:
import os
import json
import pickle
import bashlex

In [4]:
data_dir_1 = "./data/commands"

In [5]:
file_paths = []
for root, dirs, files in os.walk(data_dir_1):
    for name in files:
        if name.endswith((".json")):
            full_path = os.path.join(root, name)
            file_paths.append(full_path)

In [6]:
for path in file_paths:

    with open(path, "r") as f:
        data = f.read()
        for line in data.split("\n"):
            if len(line) == 0:
                continue
            json_data = json.loads(line)
            if "cmd" in json_data:
                dataset.append(json_data["cmd"])

In [7]:
len(dataset)

21089

https://huggingface.co/datasets/spignelon/bash_history


In [8]:
data_dir_2 = "./data/bash.txt"

In [9]:
with open(data_dir_2, "r") as f:
    data = f.read()
    lines = data.split("\n")
    dataset.extend(lines)

In [10]:
len(dataset)

202059

https://huggingface.co/datasets/gayathrimanoj/dataset_shell


In [11]:
data_dir_3 = "./data/train.jsonl"
data_dir_4 = "./data/test.jsonl"

In [12]:
with open(data_dir_3, "r") as f:
    data = f.read()
    lines = data.split("\n")
    for line in lines:
        if len(line) == 0:
            continue
        json_data = json.loads(line)
        text = json_data["text"]
        text_split = text.split("[/INST]")[1]
        res = text_split.rstrip(" </s>")
        if res == "Invalid command":
            continue
        if res == "Invalid command.":
            continue
        if res == "Invalid Command":
            continue
        if res == "Invalid Command.":
            continue
        dataset.append(res)

In [13]:
with open(data_dir_4, "r") as f:
    data = f.read()
    lines = data.split("\n")
    for line in lines:
        if len(line) == 0:
            continue
        json_data = json.loads(line)
        text = json_data["text"]
        text_split = text.split("[/INST]")[1]
        res = text_split.rstrip(" </s>")
        if res == "Invalid command":
            continue
        if res == "Invalid command.":
            continue
        if res == "Invalid Command":
            continue
        if res == "Invalid Command.":
            continue
        dataset.append(res)

In [14]:
print(len(dataset))

202601


https://huggingface.co/datasets/byroneverson/shell-cmd-instruct


In [15]:
data_dir_5 = "./data/train (1).jsonl"

In [16]:
with open(data_dir_5, "r") as f:
    data = f.read()
    for line in data.split("\n"):
        if len(line) == 0:
            continue
        json_data = json.loads(line)
        if "cmd" in json_data:
            dataset.append(json_data["cmd"])

In [17]:
print(len(dataset))

203101


https://www.kaggle.com/datasets/thuyngandao/bashlogs


In [18]:
data_dir_6 = "./data/bash_logs"

In [19]:
file_paths = []
for root, dirs, files in os.walk(data_dir_6):
    for name in files:
        full_path = os.path.join(root, name)
        file_paths.append(full_path)

In [20]:
for path in file_paths:

    with open(path, "r") as f:
        data = f.read()
        for line in data.split("\n"):
            if len(line) == 0:
                continue
            dataset.append(line)

In [21]:
print(len(dataset))

407365


Own bash history


In [22]:
data_dir_6 = "./data/own_bash_history.txt"

In [23]:
own_commands = []
with open(data_dir_6, "r") as f:
    data = f.read().split("\n")
    for line in data:
        if ";" in line:
            dataset.append(line.split(";", 1)[1])

In [24]:
print(len(dataset))

411492


# Preprocessing


In [25]:
for i, cmd in enumerate(dataset):
    dataset[i] += "\n"
    dataset[i] = dataset[i].lstrip()

In [30]:
validated = []
for cmd in dataset:
    try:
        parsed = bashlex.parse(cmd)
        validated.append(cmd)
    except:
        pass

In [31]:
len(validated)

395723

In [32]:
from collections import Counter

In [33]:
commands = []
for cmd in validated:
    command = cmd.split(" ")[0].rstrip("\n")
    commands.append(command)

In [34]:
counted = Counter(commands)

In [35]:
sorted_dict = dict(sorted(counted.items(), key=lambda item: item[1], reverse=True))

In [36]:
# with open("commands.txt", "w") as fp:
#    fp.write("\n".join(list(sorted_dict)))

Manually select most relevant commands => filter out trash commands and random data


In [37]:
with open("commands.txt", "r") as fp:
    sorted_dict = fp.read()

In [38]:
sorted_dict = sorted_dict.split("\n")

In [39]:
selected_commands = []
for cmd in validated:
    if cmd.split(" ")[0] in sorted_dict:
        selected_commands.append(cmd)

In [40]:
len(selected_commands)

251874

In [41]:
filtered_commands = []
for cmd in selected_commands:
    if len(cmd) <= 60:
        filtered_commands.append(cmd)

In [42]:
len(filtered_commands)

239267

Shuffle data to get meaningful validation set


In [47]:
# https://stackoverflow.com/questions/46447949/is-there-a-function-in-python-that-shuffle-data-by-data-blocks
import random

blocksize = 3500  # keep some bash history context

blocks = [
    filtered_commands[i : i + blocksize] for i in range(0, len(filtered_commands), blocksize)
]

random.shuffle(blocks)

data = [b for bs in blocks for b in bs]

In [48]:
len(data)

219247

In [49]:
concatinated = "".join(data)
with open("dataset.txt", "w") as fp:
    fp.write(concatinated)

In [51]:
chars = sorted(list(set(concatinated)))

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

with open("stoi", "wb") as fp:
    pickle.dump(stoi, fp)

with open("itos", "wb") as fp:
    pickle.dump(itos, fp)